# 스파크 고급 분석 6장

## 숨은 의미 분석으로 위키백과 이해하기

In [1]:
%%configure -f
{
    "proxyUser": "hduser",
    "driverMemory": "4000M", 
    "conf": {"spark.jars.packages": "graphframes:graphframes:0.3.0-spark2.0-s_2.11",
             "spark.master": "local[2]",
             "spark.jars": "hdfs://localhost:54310/jars/ch06-lsa-2.0.0-jar-with-dependencies.jar",
             "spark.sql.crossJoin.enabled": "true"}
}

In [145]:
import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import edu.umd.cloud9.collection.wikipedia.WikipediaPage
import edu.umd.cloud9.collection.wikipedia.language.EnglishWikipediaPage
import java.util.Properties

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}

import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer

import org.apache.spark.mllib.linalg.{Vectors, Vector => MLLibVector, SingularValueDecomposition, Matrix, Matrices}
import org.apache.spark.ml.linalg.{Vector => MLVector}
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix => BDenseMatrix, SparseVector => BSparseVector}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import edu.umd.cloud9.collection.wikipedia.WikipediaPage
import edu.umd.cloud9.collection.wikipedia.language.EnglishWikipediaPage
import java.util.Properties
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.mllib.linalg.{Vectors, Vector=>MLLibVector, SingularValueDecomposition, Matrix, Matrices}
import org.apache.spark.ml.linalg.{Vector=>MLVector}
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatri

In [3]:
val base = "hdfs://localhost:54310/wikipedia/"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

base: String = hdfs://localhost:54310/wikipedia/


## 데이터 준비 

* XML parsing하여 page별  seq of topic word 

In [4]:
val path = base + "wikipedia.xml"

@transient val conf = new Configuration()
conf.set(XMLInputFormat.START_TAG_KEY, "<page>")
conf.set(XMLInputFormat.END_TAG_KEY, "</page>")
val kvs = spark.sparkContext.newAPIHadoopFile(path, classOf[XMLInputFormat], classOf[LongWritable], classOf[Text], conf)
val rawXml = kvs.map(_._2.toString).toDS()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path: String = hdfs://localhost:54310/wikipedia/wikipedia.xml
conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
kvs: org.apache.spark.rdd.RDD[(org.apache.hadoop.io.LongWritable, org.apache.hadoop.io.Text)] = hdfs://localhost:54310/wikipedia/wikipedia.xml NewHadoopRDD[0] at newAPIHadoopFile at <console>:44
rawXml: org.apache.spark.sql.Dataset[String] = [value: string]


In [5]:
/**
* Returns a (title, content) pair.
*/
def wikiXmlToPlainText(pageXml: String): Option[(String, String)] = {
    val page = new EnglishWikipediaPage()

    // Wikipedia has updated their dumps slightly since Cloud9 was written, so this hacky replacement is sometimes
    // required to get parsing to work.
    val hackedPageXml:String = pageXml.replaceFirst(
        "<text bytes=\"\\d+\" xml:space=\"preserve\">", "<text xml:space=\"preserve\">")

    WikipediaPage.readPage(page, hackedPageXml)
    
    if (page.isEmpty || !page.isArticle || page.isRedirect || page.isDisambiguation || page.getTitle.contains("(disambiguation)")) {
        None
    } else {
      Some((page.getTitle, page.getContent))
    }
}

val docTexts = rawXml.filter( _ != null ).flatMap(wikiXmlToPlainText)
docTexts.head

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wikiXmlToPlainText: (pageXml: String)Option[(String, String)]
docTexts: org.apache.spark.sql.Dataset[(String, String)] = [_1: string, _2: string]
res8: (String, String) =
(Energy,"Energy








In physics, energy is the quantitative property that must be transferred to an object in order to perform work on, or to heat, the object. The second law of thermodynamics imposes limitations on the capacity of a system to transfer energy by performing work, since some of the system's energy might necessarily be   in the form of heat instead. See e.g.     Energy is a conserved quantity; the law of conservation of energy states that energy can be converted in form, but not created or destroyed. The SI unit of energy is the joule, which is the energy transferred to an object by the work of moving it a distance of 1 metre against a force of 1 newton.Common forms of energy include the kinetic energy of a moving object, the potential energy st...

### 표제어 추출, 단어 빈도-역빈도(TF-IDF) 계산하기

* stemmization 반영
* stopword.txt로 불용어 처리
* CountVectorizer로 TF
* IDF 기능으로 TF-IDF 계산

In [37]:
def createNLPPipeline(): StanfordCoreNLP = {
    val props = new Properties()
    props.put("annotators", "tokenize, ssplit, pos, lemma")
    new StanfordCoreNLP(props)    
}

def plainTextToLemmas(text:String, stopWords: Set[String], pipeline: StanfordCoreNLP): Seq[String] = {
        
    val doc = new Annotation(text)
    
    pipeline.annotate(doc)
    
    val lemmas = new ArrayBuffer[String]()
    val sentences = doc.get(classOf[SentencesAnnotation])
    for (sentence <- sentences.asScala; 
         token <- sentence.get(classOf[TokensAnnotation]).asScala) {
        val lemma = token.get(classOf[LemmaAnnotation])
        if (lemma.length > 2 && !stopWords.contains(lemma)) {
            lemmas += lemma.toLowerCase
        }
    }
    lemmas
}

/**
* Returns a document-term matrix where each element is the TF-IDF of the row's document and
* the column's term.
*
* @param docTexts a DF with two columns: title and text
* @return : DocTermMatrix,  termIds(voca) docIds(id -> title), termIdfs (단어별 IDF)
*/
def documentTermMatrix(docTexts: Dataset[(String, String)], stopWordsFile: String, numTerms: Int)
    : (DataFrame, Array[String], Map[Long, String], Array[Double]) = {
        
    val stopWords = sc.textFile(base + "stopwords.txt").collect.toSet
    val bStopWords = sc.broadcast(stopWords)    

    // document's title -> seq of lemma
    val terms: Dataset[(String, Seq[String])] = 
        docTexts.mapPartitions { iter =>
            val pipeline = createNLPPipeline()
            iter.map { case (title, contents) =>
                (title, plainTextToLemmas(contents, bStopWords.value, pipeline))
            }
        }
 
    val termsDF = terms.toDF("title", "terms").withColumn("size", size($"terms")).filter($"size" > 1)
    
    val countVectorizer = new CountVectorizer().
        setInputCol("terms").
        setOutputCol("termFreqs").
        setVocabSize(numTerms)
        
    val vocaModel = countVectorizer.fit(termsDF)
    val termIds: Array[String] = vocaModel.vocabulary
        
    val docTermFreqs:DataFrame = vocaModel.transform(termsDF)
    
    // 문서 title -> unique id
    val docIds = termsDF.rdd.
        map( _.getString(0)).
        zipWithUniqueId().
        map( _.swap).
        collect().toMap
        
    val idf = new IDF().
        setInputCol("termFreqs").
        setOutputCol("tfidfVec")
    val idfModel = idf.fit(docTermFreqs)
        
    val docTermMatrix:DataFrame = idfModel.transform(docTermFreqs)        
        
    (docTermMatrix, termIds, docIds, idfModel.idf.toArray)
}


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

createNLPPipeline: ()edu.stanford.nlp.pipeline.StanfordCoreNLP
plainTextToLemmas: (text: String, stopWords: Set[String], pipeline: edu.stanford.nlp.pipeline.StanfordCoreNLP)Seq[String]
documentTermMatrix: (docTexts: org.apache.spark.sql.Dataset[(String, String)], stopWordsFile: String, numTerms: Int)(org.apache.spark.sql.DataFrame, Array[String], Map[Long,String], Array[Double])


In [36]:
val stopWordFilePath = base + "stopwords.txt"

val (docTermMatrix, termIds, docIds, termIdfs) = documentTermMatrix(docTexts, stopWordFilePath, 20000)
docTermMatrix.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

documentTermMatrix: (docTexts: org.apache.spark.sql.Dataset[(String, String)], stopWordsFile: String, numTerms: Int)(org.apache.spark.sql.DataFrame, Array[String], Map[Long,String], Array[Double])
stopWordFilePath: String = hdfs://localhost:54310/wikipedia/stopwords.txt
docTermMatrix: org.apache.spark.sql.DataFrame = [title: string, terms: array<string> ... 3 more fields]
termIds: Array[String] = Array(-rrb-, -lrb-, -rcb-, -lcb-, point, space, line, can, state, geometry, use, government, group, one, also, two, plane, frac, mathcal, form, right, set, energy, give, surface, angle, system, time, vector, define, may, example, function, case, see, emergency, shape, -lsb-, -rsb-, theory, call, cdot, metric, first, include, number, field, coordinate, public, follow, triangle, term, universe, distance, phi, model, area, image, theorem, method, operatorname, symmetry, equation, general, structure, end, base, unit, map, object, axis, parallel, circle, mathbf, minkowski, order, many, law, mathbb,

In [41]:
docTermMatrix.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+--------------------+--------------------+
|               title|               terms|size|           termFreqs|            tfidfVec|
+--------------------+--------------------+----+--------------------+--------------------+
|              Energy|[energy, physics,...|3821|(16310,[0,1,2,3,4...|(16310,[0,1,2,3,4...|
|            Universe|[universe, univer...|5449|(16310,[0,1,2,3,4...|(16310,[0,1,2,3,4...|
|       Ambient space|[ambient, space, ...| 137|(16310,[0,1,2,3,5...|(16310,[0,1,2,3,5...|
|          Superspace|[superspace, supe...|1191|(16310,[0,1,2,3,4...|(16310,[0,1,2,3,4...|
|     Geometry Center|[geometry, center...| 364|(16310,[0,1,5,7,8...|(16310,[0,1,5,7,8...|
|          Dehn plane|[dehn, plane, geo...| 248|(16310,[0,1,2,3,4...|(16310,[0,1,2,3,4...|
|Complex reflectio...|[complex, reflect...|1932|(16310,[0,1,2,3,5...|(16310,[0,1,2,3,5...|
|    Lipschitz domain|[lipschitz, domai...| 289|(16310,[0,1,2,3,4...|(16310,[0,1,2,3,4...|

### 특이값 계산

In [70]:
// spark.mllib의 vectorRDD로 변환
def convertToOldVectors(df:DataFrame) = {
    // Old Migration Guides - MLlib : https://spark.apache.org/docs/2.2.0/ml-migration-guides.html
    
    df.select("tfidfVec").rdd.map { row =>
        Vectors.fromML(row.getAs[MLVector]("tfidfVec"))
    }
}

val k = 10
val vecRdd = convertToOldVectors(docTermMatrix)

val mat: RowMatrix = new RowMatrix(vecRdd) 
val svd = mat.computeSVD(k, computeU = true)

println(svd.s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.linalg.distributed.RowMatrix
convertToOldVectors: (df: org.apache.spark.sql.DataFrame)org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector]
k: Int = 10
vecRdd: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[401] at map at <console>:108
mat: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@d5e4cc4
svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix] =
SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@420d0714,[1428.1048884264724,1181.1939773221345,979.0249272611959,806.6422672096115,694.581972452828,655.7450400535882,638.984364351144,619.4012372879052,580.2711528203106,564.5549731288947],-0.007564968081232696   -0.03720766127091363   ... (10 total)
-0.007545286282372311   -0.03678584426480323   ...
-0.021130134071485722   -0.4544267

### 중요한 단어/문서 살펴보기

In [121]:

// 중요 의미를 갖는 최상위 단어 목록 
//  V에서 각 의미별 topN (단어, 중요도) 목록 
def topTermsInTopConcepts(
    svd: SingularValueDecomposition[RowMatrix, Matrix],
    numConcepts: Int, 
    numTerms: Int,
    termIds:Array[String]
) : Seq[Seq[(String, Double)]] = {
       
    val v = svd.V
    val arr = v.toArray
    
    // output
    val topTerms = new ArrayBuffer[Seq[(String, Double)]]()
    
    for ( i <- 0 until numConcepts ) {
        val offset = i * v.numRows       
        topTerms += arr.slice(offset, offset+v.numRows).zipWithIndex.
            sortBy(_._1).take(numTerms).map{ case (score, id) => (termIds(id), score) } 
    }
    
    topTerms
}

// 중요 의미를 갖는 최상위 문서 목록
//  U 에서 각 의미별 topN(문서, 중요도) 목록 
def topDocInTopConcepts(
    svd: SingularValueDecomposition[RowMatrix, Matrix],
    numConcepts: Int, 
    numDocs: Int,
    docIds:Map[Long,String]
) : Seq[Seq[(String, Double)]] = {
 
    val u = svd.U
    
    // output
    val topTerms = new ArrayBuffer[Seq[(String, Double)]]()
    
    for ( i <- 0 until numConcepts ) {
        topTerms += u.rows.map(row => row.toArray(i)).zipWithUniqueId().sortBy(_._1).top(numDocs).map { 
            case (score, id) => (docIds(id), score)
        }
    }
    
    topTerms
}

val topTerms:Seq[Seq[(String, Double)]]  = topTermsInTopConcepts(svd, numConcepts = 3, numTerms=3, termIds=termIds)
val topDocs:Seq[Seq[(String, Double)]]   = topDocInTopConcepts(svd, numConcepts = 3, numDocs=3, docIds=docIds)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

topTermsInTopConcepts: (svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix], numConcepts: Int, numTerms: Int, termIds: Array[String])Seq[Seq[(String, Double)]]
topDocInTopConcepts: (svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix], numConcepts: Int, numDocs: Int, docIds: Map[Long,String])Seq[Seq[(String, Double)]]
topTerms: Seq[Seq[(String, Double)]] = ArrayBuffer(ArraySeq((emergency,-0.7465818357441569), (declare,-0.349791133077605), (state,-0.1946150873436259)), ArraySeq((-rcb-,-0.45442671966358994), (-lcb-,-0.45046351338292334), (anatomy,-0.15676855801555575)), ArraySeq((-rcb-,-0.13117486016758945), (-lcb-,-0.1301037087462227), (mathcal,-0.051971094231352086)))
topDocs: Seq[Seq[(String, Double)]] = ArrayBuffer(ArraySeq((Manipulability ellipsoid,-2.498175128185344E-5), (Flatness (math

###  연관도 계산

* 단어와 단어 사이의 연관도 
  * V를 S를 고려해 normalize 하고 query 단어와 cosine similiary 기반 유사도 측정
    * (V_t * S) * query_v
* 문서와 문서 사이의 연관도
  * U를 S를 고려해 정규화하고 query 문서와 cosine similiaty 기반 유사도 측정

In [174]:
class LSAQueryEngine(svd: SingularValueDecomposition[RowMatrix, Matrix],
                     termIds:Array[String],
                     docIds:Map[Long,String],
                     termIdfs: Array[Double]
                    ) {

    import breeze.linalg.{DenseMatrix => BDenseMatrix, SparseVector => BSparseVector}
    
    val idTerms:Map[String, Int] = termIds.zipWithIndex.toMap
    val idDocs: Map[String, Long] = docIds.map(_.swap).toMap
    
    val VS: BDenseMatrix[Double] = multiplyByDiagonalMatrix(svd.V, svd.s)
    val normalizedVS: BDenseMatrix[Double] = rowsNormalized(VS)
    val US: RowMatrix = multiplyByDiagonalRowMatrix(svd.U, svd.s)
    val normalizedUS: RowMatrix = distributedRowsNormalized(US)
    
    // 하나의 단어와 연관된 단어들 찾기 
    def topTermsForTerm(termId:Int): Seq[(Double, Int)] =  {
        
        val rowVec = normalizedVS(termId, ::).t
        
        val termScores = (normalizedVS * rowVec).toArray.zipWithIndex
                
        termScores.sortBy(-_._1).take(10)
    }

    // 하나의 단어와 연관된 단어들 정보 출력하기
    def printTopTermsForTerm(term:String): Unit = {
        topTermsForTerm(idTerms(term)).map { case (score, id) =>
             println(termIds(id), score)
        }
    }
    
    // 하나의 문서와 연관된 문서들 찾기 
    def topDocsForDoc(docId:Long): Seq[(Double, Long)] = {
        
        val docRowArray = normalizedUS.rows.zipWithUniqueId.map(_.swap).lookup(docId).head.toArray
        val docRowVec = Matrices.dense(docRowArray.length, 1, docRowArray)
        
        val docScores = normalizedUS.multiply(docRowVec)
        
        docScores.rows.map(_.toArray(0)).zipWithUniqueId().filter(!_._1.isNaN).top(10)        
    }
    
    // 하나의 문서와 연관된 문서들 정보 출력하기
    def printTopDocsForDoc(doc:String): Unit = {
        topDocsForDoc(idDocs(doc)).map { case (score, docId) =>
            println( docIds(docId), score )
        }
    }
    
    def termsToQueryVector(terms:Seq[String]): BSparseVector[Double] = {
        val indices = terms.map(idTerms(_)).toArray
        val values = indices.map(termIdfs(_))
        new BSparseVector[Double](indices, values, idTerms.size)
    }
    
    def topDocsForTermQuery(query: BSparseVector[Double]): Seq[(Double, Long)] = {
        
        val v = svd.V
        val breezeV = new BDenseMatrix[Double](v.numRows, v.numCols, v.toArray)
        
        val termRowArr = (breezeV.t * query).toArray
        
        val termRowVec = Matrices.dense(termRowArr.length, 1, termRowArr)
        
        val docScores = US.multiply(termRowVec)
        
        docScores.rows.map(_.toArray(0)).zipWithUniqueId().top(10)
    }
    
    def printTopDocsForTermQuery(terms: Seq[String]):Unit = {
        
        val queryVector = termsToQueryVector(terms)
        val topDocs = topDocsForTermQuery(queryVector)
        
        topDocs.map { case (score, docId) =>
            println( docIds(docId), score )
        }
    
    }
    
    /**
    * Returns a matrix where each row is divided by its length.
    */
    def rowsNormalized(mat: BDenseMatrix[Double]): BDenseMatrix[Double] = {
        val newMat = new BDenseMatrix[Double](mat.rows, mat.cols)
        for (r <- 0 until mat.rows) {
            val length = math.sqrt((0 until mat.cols).map(c => mat(r, c) * mat(r, c)).sum)
            (0 until mat.cols).foreach(c => newMat.update(r, c, mat(r, c) / length))
        }
        newMat
    }
    
    
    /**
    * Returns a distributed matrix where each row is divided by its length.
    */
    def distributedRowsNormalized(mat: RowMatrix): RowMatrix = {
        new RowMatrix(
            mat.rows.map { vec =>
                val array = vec.toArray
                val length = math.sqrt(array.map(x => x * x).sum)
                Vectors.dense(array.map(_ / length))
            }
        )
    }
    
    /**
    * Finds the product of a dense matrix and a diagonal matrix represented by a vector.
    * Breeze doesn't support efficient diagonal representations, so multiply manually.
    */
    def multiplyByDiagonalMatrix(mat: Matrix, diag: MLLibVector): BDenseMatrix[Double] = {
        val sArr = diag.toArray
        new BDenseMatrix[Double](mat.numRows, mat.numCols, mat.toArray)
          .mapPairs { case ((r, c), v) => v * sArr(c) }
    }

    /**
    * Finds the product of a distributed matrix and a diagonal matrix represented by a vector.
    */
    def multiplyByDiagonalRowMatrix(mat: RowMatrix, diag: MLLibVector): RowMatrix = {
        val sArr = diag.toArray
        new RowMatrix(
            mat.rows.map { vec =>
                val vecArr = vec.toArray
                val newArr = (0 until vec.size).toArray.map(i => vecArr(i) * sArr(i))
                Vectors.dense(newArr)
            }
        )
    }    
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class LSAQueryEngine


In [175]:
val engine = new LSAQueryEngine(svd,termIds, docIds, termIdfs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

engine: LSAQueryEngine = LSAQueryEngine@1d25bafc


In [167]:
engine.printTopTermsForTerm("energy")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(energy,0.9999999999999999)
(nuclear,0.9972423093067236)
(fusion,0.9969145023661303)
(heat,0.9958377414805573)
(thermal,0.995201154296173)
(carbon,0.9944547724612467)
(chemical,0.9943576275841175)
(substance,0.992554424627008)
(potential,0.9915418851818423)
(release,0.9911710444441805)


In [168]:
engine.printTopDocsForDoc("Dual mandate")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(Dual mandate,0.9999999999999998)
(Criticism of government,0.9947118707974583)
(Caretaker government,0.9941160444452173)
(Legislative calendar,0.9892313159816131)
(Ministry (collective executive),0.9865869905145708)
(Administrative centre,0.9853372013300937)
(Outline of government,0.9803624502900843)
(Legislative session,0.979618539588496)
(Governmental accounting,0.9780341254196077)
(Center of government,0.9780082332970367)


In [176]:
engine.printTopDocsForTermQuery(Seq("quantitative", "property"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(Energy,22.70955453211654)
(Minkowski space,14.171607100078283)
(Busemann function,13.849470796186438)
(Computational anatomy,13.079551933027782)
(Universe,12.089365968679232)
(Line moiré,6.04189992759229)
(State of emergency,5.913983032091663)
(Fat object (geometry),5.878769726826015)
(Schema for horizontal dials,5.862831640520457)
(Spacetime diagram,5.853655764740569)
